In [ ]:
import autoreload
%load_ext autoreload
%autoreload 2  # Autoreload all modules

In [ ]:
import sys
import os
import importlib
import localSettings as ls
importlib.reload(ls)
print(ls.main_path)
print(ls.SAMPLE)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

## Change cell below to indicate whether doing run 3
### ISRUN3 = False implies Run1 (for now)

In [ ]:
# Doing Run3?
ISRUN3 = True

In [ ]:
import plotter
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
#import xgboost as xgb
#import nue_booster 
#importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

## Load Dataframes

In [ ]:
#This cell loads dataframes
importlib.reload(ls)

tree = "NeutrinoSelectionFilter"

BNB = ""
EXT = ""
NU  = ""
DRT = ""

if ISRUN3:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_G1_beam_good_reco2_1e19'+ls.APPEND
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G1_all_reco2'+ls.APPEND
    #EXT = 'ext'+ls.APPEND
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'+ls.APPEND
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'+ls.APPEND
    
else:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19'+ls.APPEND
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C1_all_reco2'+ls.APPEND
    #EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_all_reco2'+ls.APPEND #Run1 + Run2
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2'+ls.APPEND
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'+ls.APPEND

if ISRUN3:
    RUN = ls.RUN3
else:
    RUN = ls.RUN1
    
mc = uproot.open(ls.ntuple_path+RUN+NU+".root")[ls.fold][tree]
data = uproot.open(ls.ntuple_path+RUN+BNB+".root")[ls.fold][tree]
ext = uproot.open(ls.ntuple_path+RUN+EXT+".root")[ls.fold][tree]
dirt = uproot.open(ls.ntuple_path+RUN+DRT+".root")[ls.fold][tree]

uproot_v = [mc,ext,data,dirt]

variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category",
    "topological_score",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP","CosmicDirAll3D","CosmicIPAll3D",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "trk_llr_pid_score_v", # trk-PID score
    "trk_energy_proton_v", # track energy under proton hyp
    "trk_calo_energy_y_v", # track calo energy
    #"pi0_energy2_Y", # pi0 tagger variables
    #"_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "trk_sce_start_x_v","trk_sce_start_y_v","trk_sce_start_z_v",
    "trk_sce_end_x_v","trk_sce_end_y_v","trk_sce_end_z_v",
    "trk_mcs_muon_mom_v","trk_range_muon_mom_v", "trk_len_v",
    "pfp_generation_v","trk_distance_v","trk_theta_v",
    "trk_energy_muon","trk_energy_tot","trk_energy",
    "pfnhits","pfnunhits"
]
#make the list unique
variables = list(set(variables))
print(variables)

if not ISRUN3:
    variables.remove("_closestNuCosmicDist")
    variables.remove("crtveto")
    variables.remove("crthitpe")
    
WEIGHTS = ["weightSpline","weightTune","weightSplineTimesTune","leeweight"]#, "weightsGenie", "weightsFlux", "weightsReint"]
#WEIGHTSLEE = ["weightSpline","weightTune","weightSplineTimesTune", "leeweight"]#, "weightsGenie", "weightsFlux", "weightsReint"]
MCFVARS = ["mcf_nu_e","mcf_lep_e","mcf_actvol","mcf_nmm","mcf_nmp","mcf_nem","mcf_nep","mcf_np0","mcf_npp",
           "mcf_npm","mcf_mcshr_elec_etot","mcf_pass_ccpi0","mcf_pass_ncpi0",
           "mcf_pass_ccnopi","mcf_pass_ncnopi","mcf_pass_cccpi","mcf_pass_nccpi"]
SYSTEMATICS = []# ['weightsFlux','weightsGenie']

mc = mc.pandas.df(variables + WEIGHTS + MCFVARS + SYSTEMATICS, flatten=False)
print("MC dataframe built...")
data = data.pandas.df(variables, flatten=False)
print("Data dataframe built...")
ext = ext.pandas.df(variables, flatten=False)
print("EXT dataframe built...")
dirt = dirt.pandas.df(variables + WEIGHTS + SYSTEMATICS, flatten=False)
print("Dirt dataframe built...")

In [ ]:
#some scaling related calculations

df_v = [mc,dirt]

for i,df in enumerate(df_v):
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
    #df['weightSpline']  = df['weightSpline']  * df['weightTune']
    #df.loc[ df['npi0'] > 0, 'weightSplineTimesTune' ] = df['weightSpline'] * df['weightTune'] * 0.7 #scale down pi0s

In [ ]:
# how to get the LLR-PID value for the "track candidate" (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!

df_v = [mc,ext,data,dirt]

for i,df in enumerate(df_v):
    up = uproot_v[i] #uproot_v are the uproot objects, not DataFrames
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['trkthetacos'] = np.cos(df['trk_theta'])
    #df['FMdiffOBV'] = df['nu_flashmatch_score']-df['best_obviouscosmic_flashmatch_score']
    #df['FMdiff'] = df['nu_flashmatch_score']-df['best_cosmic_flashmatch_score']

In [ ]:
#
for i,df in enumerate(df_v):
    up = uproot_v[i]
    df["slclnhits"] = up.array("pfnhits").sum()
    df["slclnunhits"] = up.array("pfnunhits").sum()
#

In [ ]:
#Ryan's calculated columns
df_v = [mc,ext,data,dirt]

M_mu = 0.105 #GeV/c
M_p = 0.938 #GeV/c

for df in df_v:
    df['trk_p_quality_v'] = (df['trk_mcs_muon_mom_v']-df['trk_range_muon_mom_v'])/df['trk_range_muon_mom_v']
    df['trk_cos_v'] = df['trk_theta_v'].apply(lambda x: np.cos(x))
    df['reco_nu_e_range'] = df["trk_energy_muon"] + (df["trk_energy_tot"] - df["trk_energy"]) + M_mu
    df['reco_muon_e_range'] = df["trk_energy_muon"] + M_mu

In [ ]:
INTERCEPT = 0.0
SLOPE = 0.83

# define some energy-related variables
for i,df in enumerate(df_v):
    df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]
    df["reco_e_qe"] = 0.938*((df["shr_energy"]+INTERCEPT)/SLOPE)/(0.938 - ((df["shr_energy"]+INTERCEPT)/SLOPE)*(1-np.cos(df["shr_theta"])))
    df["reco_e_rqe"] = df["reco_e_qe"]/df["reco_e"]

# and a way to filter out data
for i,df in enumerate(df_v):
    df["bnbdata"] = np.zeros_like(df["shr_energy"])
    df["extdata"] = np.zeros_like(df["shr_energy"])
data["bnbdata"] = np.ones_like(data["shr_energy"])
ext["extdata"] = np.ones_like(ext["shr_energy"])

In [ ]:
# add back the cosmic category, for background only
df_v = [mc,ext,data,dirt]
for i,df in enumerate(df_v):
    df.loc[(df['category']!=1)&(df['category']!=10)&(df['category']!=11)&(df['category']!=111)&(df['slnunhits']/df['slnhits']<0.2), 'category'] = 4

In [ ]:
nue = mc.query('nu_pdg == 12 or nu_pdg == -12')
mc  = mc.query('nu_pdg == 14 or nu_pdg == -14')
#lee = nue

In [ ]:
#lee["is_signal"] = lee["category"] == 11
data["is_signal"] = data["category"] == 11
#nue["is_signal"] = nue["category"] == 11
mc["is_signal"] = mc["category"] == 11
dirt["is_signal"] = dirt["category"] == 11
ext["is_signal"] = ext["category"] == 11

#lee.loc[lee['category'] == 1, 'category'] = 111
#lee.loc[lee['category'] == 10, 'category'] = 111
#lee.loc[lee['category'] == 11, 'category'] = 111


# Beginning of plotting section

In [ ]:
import plotter
import importlib
importlib.reload(plotter)

In [ ]:
samples = {
    "mc": mc,
    "nue": nue,
    "data": data,
    "ext": ext,
    "dirt": dirt,
    #"lee": lee,
}

scaling = 1

#0304
if ISRUN3:
    weights = {
        "mc": 5.70e-03 * scaling,
        "nue": 5.70e-3 * scaling,#should be identical to numu weight, since parsed from same sample
        #"nue": 1.21e-04 * scaling, #weight when using exclusive nue sa3mple
        #"ext": 3.02E-02 * scaling, #for the combined EXT sample
        "ext": 2.52E-02 * scaling, #G1
        "dirt": 2.35e-02 * scaling,
        #"lee": 1.21e-04 * scaling,
    }
    pot = 0.763e19*scaling
else:
    weights = {
        "mc": 3.12e-02 * scaling,
        "nue": 7.73e-04 * scaling,
        "ext": 2.69E-01 * scaling, #C+D+E #C only: 1.40e-01
        "dirt": 1.26e-01 * scaling,
        #"lee": 7.73e-04 * scaling,
    }
    pot = 4.08e19*scaling


my_plotter = plotter.Plotter(samples, weights, pot=pot)
print(weights)

In [ ]:
#updated for SCE
#will swap numerical vals at end   systematic boundaries
FVx = [5,251]                      #[10,246]
FVy = [-110,110]                   #[-105,105]
FVz = [20,986]                     #[25,986]

# muon selection
QUERY = 'nslice == 1'
#if ISRUN3: QUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
#QUERY += ' and trk_len > 20'
QUERY += ' and topological_score > 0.06'
QUERY += ' and reco_nu_vtx_sce_x > FVx[0] and reco_nu_vtx_sce_x < FVx[1]'
QUERY += ' and reco_nu_vtx_sce_y > FVy[0] and reco_nu_vtx_sce_y < FVy[1]'
QUERY += ' and reco_nu_vtx_sce_z > FVz[0] and reco_nu_vtx_sce_z < FVz[1]'
QUERY += ' and ( (reco_nu_vtx_sce_z < 675) or (reco_nu_vtx_sce_z > 775) )' #avoid dead wire region
#QUERY += ' and trkpid > 0.5'
#QUERY += ' and (trkthetacos < 0.853 or trkthetacos > 0.8531)'
#QUERY += ' and bkpdg == 2212 and (trkcompl < 0.5 or trkpur < 0.5)'
if ISRUN3:
    QUERY += ' and (crtveto!=1 or crthitpe < 100.) and (_closestNuCosmicDist > 5.)' # Note: CRTs weren't installed for until after Run 1

#okay, so the track level cuts are a bit more complicated
#can't just use query. Gotta do each cut one at a time
#(variable (must end in _v), operation, value)
track_cuts = [
    ('trk_sce_start_x_v', '>', FVx[0]),
    ('trk_sce_start_x_v', '<', FVx[1]),
    ('trk_sce_start_y_v', '>', FVy[0]),
    ('trk_sce_start_y_v', '<', FVy[1]),
    ('trk_sce_start_z_v', '>', FVz[0]),
    ('trk_sce_start_z_v', '<', FVz[1]),
    ('trk_sce_end_x_v', '>', FVx[0]),
    ('trk_sce_end_x_v', '<', FVx[1]),
    ('trk_sce_end_y_v', '>', FVy[0]),
    ('trk_sce_end_y_v', '<', FVy[1]),
    ('trk_sce_end_z_v', '>', FVz[0]),
    ('trk_sce_end_z_v', '<', FVz[1]),
    ('trk_p_quality_v', '>', -0.5),
    ('trk_p_quality_v', '<', 0.5),
    ('trk_llr_pid_score_v', '>', 0.2),
    ('trk_score_v', '>', 0.8),
    ('trk_len_v', '>', 10),
    ('pfp_generation_v', '==', 2),
    ('trk_distance_v', '<', 4)
]

QUERY = QUERY.replace('FVx[0]',str(FVx[0]))
QUERY = QUERY.replace('FVy[0]',str(FVy[0]))
QUERY = QUERY.replace('FVz[0]',str(FVz[0]))
QUERY = QUERY.replace('FVx[1]',str(FVx[1]))
QUERY = QUERY.replace('FVy[1]',str(FVy[1]))
QUERY = QUERY.replace('FVz[1]',str(FVz[1]))

print ("QUERY:\n {}\n".format(QUERY))
print("track_cuts:\n{}".format(track_cuts))

In [ ]:
############################
## Make just one plot
###########################

importlib.reload(plotter)
my_plotter = plotter.Plotter(samples, weights, pot=pot)

QUERY = None
track_cuts = None

#active volume
AVx = [-1.55,254.8]
AVy = [-115.53, 117.47]
AVz = [0.1, 1036.9]

VARIABLE, BINS, RANGE, XTIT = 'trk_p_quality_v',25,(-1,5),r"MCS Quality"
#VARIABLE, BINS, RANGE, XTIT = 'reco_nu_vtx_sce_y',25,(AVy[0],AVy[1]),r"Reconstructed Muon Endpoint Y [cm]"
tag = "_presel" #tag gets added to end of .png name


fig, ax1, ax2 = my_plotter.plot_variable(
    VARIABLE,   
    query=QUERY,#+" and category != 111",
    kind="event_category",
    #kind="sample", 
    #kind="interaction",
    #kind='particle_pdg',
    draw_sys=False,
    stacksort=3,
    track_cuts = track_cuts,
    select_longest = True, #this is true by default in self._selection
    title=XTIT,
    #bins=asymm_bins,
    bins=BINS,
    range=RANGE,
)[0:3]

print("Profile likelihood: {} sigma @ {} POT".format(my_plotter.significance_likelihood,pot))
print("s/sqrt(b): {} sigma @ {} POT".format(my_plotter.significance, pot))

#ax1.set_ylim(0,40)
#ax1.set_yscale("log")
#ax1.set_ylim(0,240)
ax1.set_ylim(0,ax1.get_ylim()[1]*1.5)
#ax2.set_ylim(0.5,1.5)
#fig.savefig(ls.plots_path+RUN+"0304_numupresel\\"+VARIABLE+"_"+date_time+".pdf")
SAVEDIR = ls.plots_path+RUN+ls.SAMPLE+'\\KINEMATICS\\'
if not os.path.exists(SAVEDIR):
    os.makedirs(SAVEDIR)
#fig.savefig(SAVEDIR+VARIABLE+"_"+date_time+tag+".pdf")
plt.show()

In [ ]:
##################################
# Make several plots at once
# note the SAVEDIR before running cell
##################################

#reload helper script incase of changes
importlib.reload(plotter)
my_plotter = plotter.Plotter(samples, weights, pot=pot)

#potentially useful values
AVx = [-1.55,254.8]
AVy = [-115.53, 117.47]
AVz = [0.1, 1036.9]
M_mu = 0.105 #GeV/c

tag = "_KINEMATICS" #tag gets added to end of filename

muon_vars = [
    'topological_score',
    'contained_fraction',
    '_closestNuCosmicDist','crthitpe',
    'reco_nu_vtx_sce_x','reco_nu_vtx_sce_y','reco_nu_vtx_sce_z',
    'crtveto'
]
muon_bins = [26]*len(muon_vars)
muon_bins[-1] = 2 #crtveto has two values

muon_ranges = [
    (0,1),
    (0,1),
    (0,300),(0,300),
    (AVx[0],AVx[1]),(AVy[0],AVy[1]),(AVz[0],AVz[1]),
    (-0.5,1.5)
]
muon_titles = [
    'Topological Score',
    'Contained Fraction',
    r'Closest $\nu$-Cosmic Distance [cm]','CRT Hit PE',
    r'Reco $\nu$ vertex X [cm]',r'Reco $\nu$ vertex Y [cm]',r'Reco $\nu$ vertex Z [cm]',
    'CRT Veto'
]

#QUERY = 'nslice == 1' #equivalent to QUERY == None
#track_cuts = None #None, not []
print(QUERY,track_cuts)

print("VARIABLE\tBINS\tRANGE\tTITLE")
for (VARIABLE,BINS,RANGE,XTIT) in zip(muon_vars,muon_bins,muon_ranges,muon_titles):
    print("{}\t{}\t{}\t{}".format(VARIABLE,BINS,RANGE,XTIT))
    fig, ax1, ax2 = my_plotter.plot_variable(
        VARIABLE,   
        query=QUERY,#+" and category != 111",
        kind="event_category",
        #kind="sample", 
        #kind="interaction",
        #kind='particle_pdg',
        draw_sys=False,
        stacksort=3,
        track_cuts = track_cuts,
        select_longest = True, #this is true by default in self._selection
        title=XTIT,
        #bins=asymm_bins,
        bins=BINS,
        range=RANGE,
    )[0:3]

    print("Profile likelihood: {} sigma @ {} POT".format(my_plotter.significance_likelihood,pot))
    print("s/sqrt(b): {} sigma @ {} POT".format(my_plotter.significance, pot))

    #ax1.set_ylim(0,40)
    #ax1.set_yscale("log")
    #ax1.set_ylim(0,240)
    ax1.set_ylim(0,ax1.get_ylim()[1]*1.5)
    #ax2.set_ylim(0.5,1.5)
    #fig.savefig(ls.plots_path+RUN+"0304_numupresel\\"+VARIABLE+"_"+date_time+".pdf")
    SAVEDIR = ls.plots_path+RUN+ls.SAMPLE+'\\KINEMATICS\\'
    if not os.path.exists(SAVEDIR):
        os.makedirs(SAVEDIR)
    #fig.savefig(SAVEDIR+VARIABLE+"_"+date_time+tag+".pdf")
    plt.show()
    

In [ ]:
# create output files based on selection
uproot_v = [lee,mc,nue,ext,data,dirt]
fname_v = [NUE,NU,NUE,EXT,BNB,DRT]

NU_Q = "~(abs(nu_pdg) == 12 & ccnc == 0)"
NU_Q += " & ~(mcf_np0==1 & mcf_nmp==0 & mcf_nmm==0 & mcf_nem==0 & mcf_nep==0)"
NU_Q += " & ~(mcf_pass_ccpi0==1)"
NU_Q += " & ~(mcf_pass_ccnopi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_ncnopi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_cccpi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_nccpi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"

for i,name in enumerate(fname_v):
    if (i == 0):
        continue # skip LEE event. This one will be done within SBNFit
        
    df = dfbdt_v[i]
    
    fout = open(ls.ntuple_path+name+'.txt','w')
    dfsel = df.query(QUERY)
    if (i == 1):
        dfsel = dfsel.query(NU_Q)
    print ('file %s has %i selected entries'%(name,dfsel.shape[0]))
    for i,row in dfsel.iterrows():
        run = row['run']
        sub = row['sub']
        evt = row['evt']
        fout.write('%i %i %i \n'%(run,sub,evt))
    fout.close()

### Energy resolution with MCS consistency cut

In [ ]:
######################################
## Energy res with and without MCS cut
#######################################
importlib.reload(plotter)
my_plotter = plotter.Plotter(samples, weights, pot=pot)

SAVEDIR = ls.plots_path+RUN+ls.SAMPLE+'\\EnergyRes-Pquality\\'

track_cuts = [
    ('trk_sce_start_x_v', '>', FVx[0]),
    ('trk_sce_start_x_v', '<', FVx[1]),
    ('trk_sce_start_y_v', '>', FVy[0]),
    ('trk_sce_start_y_v', '<', FVy[1]),
    ('trk_sce_start_z_v', '>', FVz[0]),
    ('trk_sce_start_z_v', '<', FVz[1]),
    ('trk_sce_end_x_v', '>', FVx[0]),
    ('trk_sce_end_x_v', '<', FVx[1]),
    ('trk_sce_end_y_v', '>', FVy[0]),
    ('trk_sce_end_y_v', '<', FVy[1]),
    ('trk_sce_end_z_v', '>', FVz[0]),
    ('trk_sce_end_z_v', '<', FVz[1]),
    ('trk_p_quality_v', '>', -0.5),
    ('trk_p_quality_v', '<', 0.5),
    ('trk_llr_pid_score_v', '>', 0.2),
    ('trk_score_v', '>', 0.8),
    ('trk_len_v', '>', 10),
    ('pfp_generation_v', '==', 2),
    ('trk_distance_v', '<', 4)
]

#track_cuts = []

fig,axes = plt.subplots(1,2, figsize=(10,5),
                       sharey=True)


print("making first plot...")
_,_,image = my_plotter.plot_2d_oneplot('nu_e','reco_e_range',QUERY,
                track_cuts = track_cuts,
                range_x = (0.15,1.5),
                range_y = (0.15,1.5),
                bins_x = 50,
                bins_y = 50,
                range_z = [0,1],
                figure = fig,
                axis = axes[0],
              )

axes[0].set_xlabel(r"$E_{True}^{\nu_{\mu}}$")    
axes[0].set_ylabel(r"$E_{range}^{\nu_{\mu}}$ [GeV]")
axes[0].set_title("Selected With MCS Quality Cut")

#invert momentum consistency cut
track_cuts.remove(('trk_p_quality_v', '>', -0.5))
track_cuts.remove(('trk_p_quality_v', '<', 0.5))
#track_cuts = []
track_cuts.append(('trk_p_quality_v', ['<','>'], [-0.5,0.5]))

print("making second plot...")
_,_,image = my_plotter.plot_2d_oneplot('nu_e','reco_e_range',QUERY,
                track_cuts = track_cuts,
                range_x = (0.15,1.5),
                range_y = (0.15,1.5),
                bins_x = 50,
                bins_y = 50,
                #range_z = [0,1],
                figure = fig,
                axis = axes[1],
              )

axes[1].set_xlabel(r"$E_{True}^{\nu_{\mu}}$")    
axes[1].set_title("Rejected by MCS Cut")
fig.colorbar(image, ax=axes)


fig.savefig(SAVEDIR+"EnergyRes_"+date_time+"_subplots_alltrackcuts.pdf")